In [11]:
import sys
import os

# Get the absolute path of the parent's parent directory
parent_parent_dir = os.path.abspath('../../')

# Add it to the Python path
sys.path.insert(0, parent_parent_dir)

In [12]:
from IPython.display import display
import pandas as pd
from openssa.core.rag_ooda.rag_ooda import RagOODA
from openssa.core.rag_ooda.resources.standard_vi.standard_vi import load_standard_vi
from openssa.core.rag_ooda.resources.dense_x.dense_x import load_dense_x

In [13]:
# supporting class and function
class RagStrategy:
    dense = "dense"
    standard_vi = "standard_vi"


class RagUseCase:
    m290 = "m290"
    l316 = "l316"


DATA_FOLDER_PATH = "../../examples/data"


def get_dirs(use_case: str, strategy: str) -> tuple:
    cache_dir = f"{DATA_FOLDER_PATH}/cache/{strategy}/{use_case}/indexes"
    data_dir = f"{DATA_FOLDER_PATH}/docs/{use_case}"
    nodes_cache_path = f"{DATA_FOLDER_PATH}/cache/{strategy}/{use_case}/nodes_dict.json"
    return cache_dir, data_dir, nodes_cache_path

def load_standard_vector_index():
    use_case = RagUseCase.m290
    strategy = RagStrategy.standard_vi
    cache_dir, data_dir, _ = get_dirs(use_case, strategy)
    print(f"Loading resources... {use_case} with strategy {strategy} ")
    return load_standard_vi(data_dir, cache_dir)


def load_dense_x_index():
    strategy = RagStrategy.dense
    use_case = RagUseCase.m290
    cache_dir, data_dir, nodes_cache_path = get_dirs(use_case, strategy)
    print(f"Loading resources... {use_case} with strategy {strategy} ")
    return load_dense_x(data_dir, cache_dir, nodes_cache_path)

In [14]:
# load resources

rag_standard_vi = load_standard_vector_index()
print("standard vector index loaded.")

# rag_dense_x = load_dense_x_index()
# print("dense_x indexes loaded.")

resources = [rag_standard_vi]
rag_ooda = RagOODA(resources=resources)

Loading resources... m290 with strategy standard_vi 


2024-01-04 15:26:28.521 | DEBUG    | openssa.utils.utils:wrapper:339 - Function load_standard_vi took 2.6162930720020086 seconds


standard vector index loaded.


In [15]:
questions = [
        "do I need heat treatment for the 316L with M 290?",
        "Which steel can I print?",
        "can I print a part 50 cm long in the M 290?",
        "Is it preferable to use nitrogen when printing with titanium?",
        # "List all the materials I can print with?",
        # "What are the calibration steps for a new build?",
        # "what are the size limits?",
        # "what are the parts size limits for the M 290?",
    ]

In [17]:
# generate results

svi_answers = []
ro_answers = []
# dense_x_answers = []

for q in questions:
    print(f"\nQuestion: {q}\n")
    stand_vi_answer = rag_standard_vi.query_engine.query(q).response
    svi_answers.append(stand_vi_answer)
    # dense_x_answer = rag_dense_x.query_engine.query(q)
    # dense_x_answers.append(dense_x_answer)
    rag_ooda_answer = rag_ooda.chat(q)
    ro_answers.append(rag_ooda_answer)

df = pd.DataFrame(  # noqa: PD901
    {
        "question": questions,
        "standard vector index": svi_answers,
        # "dense x": dense_x_answers,
        "rag ooda": ro_answers,
    }
)
df.to_csv(f"{DATA_FOLDER_PATH}/notebook_qa_comparion.csv", index=False)

display(df)


Question: do I need heat treatment for the 316L with M 290?



2024-01-04 15:30:24.882 | DEBUG    | openssa.utils.utils:wrapper:339 - Function ContextValidator.execute took 1.7562697399989702 seconds
2024-01-04 15:30:27.143 | DEBUG    | openssa.utils.utils:wrapper:339 - Function SynthesizingAgent.execute took 2.260532272979617 seconds
2024-01-04 15:30:27.144 | DEBUG    | openssa.utils.utils:wrapper:339 - Function RagOODA.chat took 4.394045699969865 seconds



Question: Which steel can I print?



2024-01-04 15:30:34.652 | DEBUG    | openssa.utils.utils:wrapper:339 - Function ContextValidator.execute took 3.998179643007461 seconds
2024-01-04 15:30:42.158 | DEBUG    | openssa.utils.utils:wrapper:339 - Function SynthesizingAgent.execute took 7.504504274053033 seconds
2024-01-04 15:30:42.159 | DEBUG    | openssa.utils.utils:wrapper:339 - Function RagOODA.chat took 11.93142954702489 seconds



Question: can I print a part 50 cm long in the M 290?



...


assistant: Tool results for question can I print a part 50 cm long in the M 290? is: Yes, you can print a part 50 cm long in the M 290.
Event: ooda-maintask-observe, Data: {'thought': 'The specifications of the M 290 3D printer do not provide information about its maximum build size, making it difficult to ascertain whether it can accommodate a part of 50 cm in length.', 'tool_results': 'Yes, you can print a part 50 cm long in the M 290.', 'uuid': '73896555-e2db-49e2-b4c5-f921929d3116', 'tool_executions': "{'tool_name': 'research_documents', 'parameters': {'task': 'can I print a part 50 cm long in the M 290?'}}"}
Event: ooda-maintask-orient, Data: {'thought': "Given the lack of specific information regarding the maximum build size, it is challenging to determine if the M 290 can print a part 50 cm long. It may be necessary to consult the manufacturer's documentation or technical support for clarification.", 'tool_results': {}, 'uuid': '73896555-e2db-49e2-b4c5-f921929d3116'}
Event: ood

2024-01-04 15:31:58.357 | DEBUG    | openssa.utils.utils:wrapper:339 - Function Solver.synthesize_result took 41.04632716497872 seconds
2024-01-04 15:31:58.359 | DEBUG    | openssa.utils.utils:wrapper:339 - Function RagOODA.chat took 74.53510849596933 seconds


Event: task_result, Data: {'response': "The conversation regarding the printing of a 50 cm long part using an EOS M 290 3D printer includes conflicting information. Initially, there was ambiguity since no specific details about the maximum build size of the EOS M 290 were provided. When the user asked whether a part 50 cm long could be printed, the response was that it is possible to print a part 50 cm long in the M 290. However, no specific dimensions were provided to support this claim.\n\nIn the discussion, there is a mention of researching the build size of the M 290 printer, which indicates that the appropriate next step would involve finding out the printer's build volume capabilities to accurately assess if a 50 cm long part is printable. However, the actual build volume specifications of the EOS M 290 are not detailed within the conversation.\n\nWithout the explicit dimensions of the printer's build volume, there are two contrasting statuses: one suggesting that printing a 50 c

2024-01-04 15:32:02.878 | DEBUG    | openssa.utils.utils:wrapper:339 - Function ContextValidator.execute took 1.8199073860305361 seconds
2024-01-04 15:32:10.866 | DEBUG    | openssa.utils.utils:wrapper:339 - Function SynthesizingAgent.execute took 7.987462755991146 seconds
2024-01-04 15:32:10.868 | DEBUG    | openssa.utils.utils:wrapper:339 - Function RagOODA.chat took 10.239760412019677 seconds


,question,standard vector index,rag ooda
0,do I need heat treatment for the 316L with M 290?,"Yes, heat treatment is required for the 316L w...","Yes, heat treatment is required for 316L when ..."
1,Which steel can I print?,You can print EOS CaseHardeningSteel 20MnCrZr9...,You can print with the following steels using ...
2,can I print a part 50 cm long in the M 290?,"Yes, you can print a part 50 cm long in the M ...",The conversation regarding the printing of a 5...
3,Is it preferable to use nitrogen when printing...,"Yes, it is preferable to use nitrogen when pri...","No, it is not preferable to use nitrogen when ..."
